In [1]:
# Import source

using Test
using Plots
pyplot()

using Distributions
using MCMCDiagnostics
using StatsBase

cd("../Source")
src_path = "../Source/"
include(src_path*"types.jl")
include(src_path*"functions_msc.jl")
include(src_path*"functions_neighbors.jl")
include(src_path*"functions_types.jl")
include(src_path*"functions_energy.jl")
include(src_path*"functions_mc.jl")
#@everywhere include(src_path*"functions_thermalization.jl")
#@everywhere include(src_path*"functions_observables.jl")
#@everywhere include(src_path*"functions_symmetries.jl")
include(src_path*"Grid/xy_energy.jl")


cd("../Notebooks/")

In [7]:
include(src_path*"Grid/xy_energy.jl")

ΔE (generic function with 2 methods)

In [18]:
include("../Source/functions_energy.jl")

ΔE (generic function with 1 method)

In [2]:
function testEnDiff(ψ₀::State, ψ::State, dE::Float64)
    E₀ = E(ψ₀)
    E′ = E(ψ)
    if E₀ > E′
        same = isapprox(E′-dE, E₀; atol=0, rtol=1e-13)
    else
        same = isapprox(E′, E₀+dE; atol=0, rtol=1e-13)
    end
    if !same
        println("|(E′-E₀)-dE|/max(E′-E₀, dE) = $(abs((E′-E₀)-dE)/max(E′-E₀, dE))")
        println("E₀\t\t\tΔE\t\t\tE′-E₀\t\t\t|ΔE-E₁-E₂|")
        println("$E₀\t$dE\t$(E′-E₀)\t$(abs(dE - (E′-E₀)))")
    end
    return same
end

testEnDiff (generic function with 2 methods)

In [16]:
g = 0.3; ν = 0.3; L = 12; T = 4.3; κ₅ = 1.0; f = 1.4/L#0.12/L
β = 1/T
sim = Controls(0.4, 0.4, 0.18)
# Create system
syst = SystConstants(L, L, 1/g^2, ν, κ₅, f, β)
ψ₀ = State(1, syst)
ψ = copy(ψ₀)
dE = 0.0
for i = 1:5000
    dE += mcSweepEn!(ψ, sim)
end
E′ = E(ψ)
E⁰ = E(ψ₀)
println("dE =\t$(dE)\nE-E₀ =\t$(E(ψ)-E(ψ₀))\nE₀ =\t$(E(ψ₀))\n|dE - (E-E₀)| =\t$(abs(dE-(E′-E⁰)))")

dE =	13751.968148355956
E-E₀ =	13751.968148355969
E₀ =	2975.0040104385125
|dE - (E-E₀)| =	1.2732925824820995e-11


In [17]:
println("\nTesting Energy difference function ΔE\n----------------------------------------------------------------")
# Test case. We make a random 3x3 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

L = 5
syst = SystConstants(L, L, 1/g^2, ν, κ₅, f, β)
ψ₂ = State(1, syst)
ϕ′ = LatticeSite() # Get random lattice site
ψ₁ = copy(ψ₂)
@test ψ₁.nb[3,2,2].ϕᵣ₊₂.u⁺ == ψ₁.lattice[2,2,2].u⁺
@test ψ₁.nb[3,2,2].ϕᵣ₊₂ == ψ₁.lattice[2,2,2]
set!(ψ₁.lattice[2,2,2], ϕ′)
@test ψ₁.nb[3,2,2].ϕᵣ₊₂.u⁺ == ϕ′.u⁺

dE = ΔE(ϕ′,ψ₂.lattice[2,2,2],ψ₂.nb[2,2,2],ψ₂.nnb[2,2,2],ψ₂.nnnb[2,2,2], 2,ψ₂.consts)
println("Checking that ΔE and E get same result")
println(@test testEnDiff(ψ₂, ψ₁, dE))
#println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13))



Testing Energy difference function ΔE
----------------------------------------------------------------
Checking that ΔE and E get same result
Test Passed


In [19]:
g = 0.3; ν = 0.3; L = 12; T = 4.12; κ₅ = 1.0; f = 1.4/L#0.12/L
β = 1/T
# Create system
syst = SystConstants(L, L, 1/g^2, ν, κ₅, f, β)
ψ₀ = State(2, syst)
println("Checking that ΔE gives same result as E'-E on all different sites of the lattice")
ψ = copy(ψ₀)
for z_pos = 1:L, h_pos = 1:L, v_pos = 1:L
    ψ = copy(ψ₀)
    ϕ′ = LatticeSite()
    dE = ΔE(ϕ′, ψ₀.lattice[v_pos,h_pos,z_pos],ψ₀.nb[v_pos,h_pos,z_pos],ψ₀.nnb[v_pos,h_pos,z_pos],ψ₀.nnnb[v_pos,h_pos,z_pos],h_pos,ψ₀.consts)
    set!(ψ.lattice[v_pos,h_pos,z_pos], ϕ′)
    if !testEnDiff(ψ₀, ψ, dE)
        println("[$(v_pos), $(h_pos), $(z_pos)]")
    end
    #@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13)
end
println(@test isapprox(E(ψ)-dE, E(ψ₀); atol=0, rtol=1e-13))



Checking that ΔE gives same result as E'-E on all different sites of the lattice
Test Passed


In [21]:
println("\nTesting Energy difference function ΔE\n----------------------------------------------------------------")
# Test case. We make a random 5x5 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

L = 5
syst = SystConstants(L, L, 1/g^2, ν, κ₅, f, β)
ψ₂ = State(2,syst)
site = LatticeSite() # Get random lattice site
ψ₁ = copy(ψ₂)
@test ψ₁.nb[3,2,1].ϕᵣ₊₂.u⁺ == ψ₁.lattice[2,2,1].u⁺
@test ψ₁.nb[3,2,1].ϕᵣ₊₂ == ψ₁.lattice[2,2,1]
set!(ψ₁.lattice[2,2,1], site)
@test ψ₁.nb[3,2,1].ϕᵣ₊₂.u⁺ == site.u⁺

dE = ΔE(site,ψ₂.lattice[2,2,1],ψ₂.nb[2,2,1],ψ₂.nnb[2,2,1],ψ₂.nnnb[2,2,1], 2,ψ₂.consts)
println("Checking that ΔE and E get the same result")
println("ΔE\t\t\tE₁-E₂")
println("$dE\t$(E(ψ₁)-E(ψ₂))")
println(dE/(E(ψ₁)-E(ψ₂)))
println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13))


Testing Energy difference function ΔE
----------------------------------------------------------------
Checking that ΔE and E get the same result
ΔE			E₁-E₂
-2556.254969927242	-2556.254969927133
1.0000000000000426
Test Passed


In [23]:
# -------------------------------------------------------------------------------------------------
# Function that uses that changes one lattice site at the time in the whole lattice of the State and
# calculates the difference in energy between using E(ψ′)-E(ψ) or using ΔE to calculate energy differences.
function testEnDiff(ψ₀::State)
    println("Checking that ΔE gives same result as E'-E on all different sites of the lattice")
    L₃ = ψ₀.consts.L₃
    L = ψ₀.consts.L
    ψ = copy(ψ₀)
    dE = 0
    for z_pos = 1:L₃, h_pos = 1:L, v_pos = 1:L
        ψ = copy(ψ₀)
        ϕ′ = LatticeSite()
        dE = ΔE(ϕ′, ψ₀.lattice[v_pos,h_pos,z_pos],ψ₀.nb[v_pos,h_pos,z_pos],ψ₀.nnb[v_pos,h_pos,z_pos],ψ₀.nnnb[v_pos,h_pos,z_pos],h_pos,ψ₀.consts)
        set!(ψ.lattice[v_pos,h_pos,z_pos], ϕ′)
        if !testEnDiff(ψ₀, ψ, dE)
            println("[$(v_pos), $(h_pos), $(z_pos)]")
        end
        #@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13)
    end
    return @test isapprox(E(ψ)-dE, E(ψ₀); atol=0, rtol=1e-13)
end

testEnDiff (generic function with 2 methods)

In [25]:
testEnDiff(State(1,syst))

Checking that ΔE gives same result as E'-E on all different sites of the lattice


Test Passed